In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
import re

In [3]:
%%bash
cp /storage/group/ibb3/default/AP_AC_genome_seqs/dovetail_Apalm/HiC_improvement/Apalm_assembly_v3.1_200911.masked.fasta \
    /storage/group/ibb3/default/convert_snpchip_coords/apal.fa

In [4]:
%%bash
source ~/.bashrc
conda activate ipyrad
bwa index /storage/group/ibb3/default/convert_snpchip_coords/apal.fa

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[bwa_index] Pack FASTA... 1.55 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=575184370, availableWord=52471840
[BWTIncConstructFromPacked] 10 iterations done. 84109298 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 157733490 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 223166834 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 281320130 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 333002930 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 378934738 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 419754994 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 456032114 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 488271314 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 516921586 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 542

In [6]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/convert_snpchip_coords/
bwa mem \
    ./apal.fa \
    probes.fasta \
    > ./probes_aln_apal.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 53589 sequences (3803707 bp)...
[M::mem_process_seqs] Processed 53589 reads in 2.784 CPU sec, 2.789 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ./apal.fa probes.fasta
[main] Real time: 3.283 sec; CPU: 2.969 sec


In [7]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/convert_snpchip_coords/
sambamba view -t 4 --sam-input \
    -F "mapping_quality >= 3 and not (unmapped or secondary_alignment or supplementary or duplicate)" \
    ./probes_aln_apal.sam > ./unique_mappings_apal_probes.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



sambamba 0.7.1
 by Artem Tarasov and Pjotr Prins (C) 2012-2019
    LDC 1.20.0 / DMD v2.090.1 / LLVM7.0.0 / bootstrap LDC - the LLVM D compiler (0.17.6)



In [8]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/
cut -f1,3,4,5,6,10,12 /storage/group/ibb3/default/convert_snpchip_coords/unique_mappings_apal_probes.sam \
    > /storage/group/ibb3/default/convert_snpchip_coords/unique_mappings_apal_probes.tab

In [3]:
probes = pd.read_csv("/storage/group/ibb3/default/convert_snpchip_coords/unique_mappings_apal_probes.tab", comment = "@", sep = "\t", names = ["probe", "contig", "start", "mapq", "cigar", "seq", "mismatch"])

In [4]:
probes = probes[probes["mapq"] > 20].reset_index(drop=True)
probes["npos"] = np.nan
for i in range(len(probes)):
    probes["npos"][i] = probes["seq"][i].find("N")
probes["new_snp_pos"] = np.nan
probes.loc[~probes["cigar"].str.contains("I|D"), "new_snp_pos"] = probes["npos"]

/tmp/ipykernel_3221775/435527431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["npos"][i] = probes["seq"][i].find("N")


In [35]:
for i in range(len(probes)):
    cigar = re.findall('(\d+)?([A-Za-z])', probes["cigar"][i])
    length = len(cigar)
    if (length < 4) and (length > 2):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ) and ( cigar[2][1] == "S" ):
            print
        elif ( cigar[0][1] == "M" ) and ( cigar[2][1] == "M" ):
            if int(cigar[0][0]) > probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
            elif int(cigar[0][0]) < probes["seq"][i].find("N"):
                if cigar[1][1] == "I":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") + int(cigar[1][0])
                elif cigar[1][1] == "D":
                    if (int(cigar[0][0]) + int(cigar[1][0])) > probes["npos"][i]:
                        pass
                    else:
                        probes["new_snp_pos"][i] = probes["seq"][i].find("N") - int(cigar[1][0])
    if (length > 3):
        if ( cigar[0][1] == "S" ) and ( cigar[1][1] == "M" ):
            if (int(cigar[0][0]) + int(cigar[1][0])) >= probes["seq"][i].find("N"):
                probes["new_snp_pos"][i] = probes["seq"][i].find("N")
    if ( cigar[0][1] == "M" ) and ( int(cigar[0][0]) >= probes["seq"][i].find("N") ):
        probes["new_snp_pos"][i] = probes["seq"][i].find("N")

/tmp/ipykernel_3453425/1691801142.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_3453425/1691801142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N")
/tmp/ipykernel_3453425/1691801142.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["seq"][i].find("N") - int(cigar[1][0]

In [36]:
probes["new_snp_pos"] = probes["start"] + probes["new_snp_pos"]

In [5]:
probes = probes[probes["new_snp_pos"].notna()].reset_index(drop=True)
#now excluding probes w/ excessive mismatch due to indels --> probes.loc[probes["new_snp_pos"].isna(), "new_snp_pos"] = probes["start"] + probes["npos"]
probes["new_snp_pos"] = probes["new_snp_pos"].astype(int)

In [6]:
recommended = pd.read_csv("/storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/recommended.txt", header = None)

In [7]:
probes = probes[probes["probe"].isin(recommended[0].tolist())].reset_index(drop=True)

In [8]:
!rm /storage/group/ibb3/default/convert_snpchip_coords/probes_in_apal.txt

In [9]:
file = open('/storage/group/ibb3/default/convert_snpchip_coords/probes_in_apal.txt','w')
for item in probes["probe"].tolist():
    file.write(item+"\n")
file.close()

In [10]:
reformat = probes[["contig", "start", "probe"]]
reformat.columns = ["contig", "start", "ID"]

In [11]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/
grep -v "^##" /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename.vcf | cut -f1-3 \
    > ./original_coords_apal.tab

In [12]:
tmp = pd.read_csv("/storage/group/ibb3/default/convert_snpchip_coords/original_coords_apal.tab", sep = "\t")

In [13]:
tmp = tmp.merge(reformat, how = "left", on = "ID")

In [14]:
tmp["contig"] = tmp["contig"].fillna("NOMATCH")
tmp["start"] = tmp["start"].fillna(0)

In [15]:
reformatted_positions = tmp[["contig", "start", "ID"]]
reformatted_positions.columns = ["#CHROM", "POS", "ID"]

In [16]:
reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)

/tmp/ipykernel_3221775/2815286365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reformatted_positions["POS"] = reformatted_positions["POS"].astype(int)


In [17]:
reformatted_positions.set_index("#CHROM").to_csv("/storage/group/ibb3/default/convert_snpchip_coords/remapped_coords_apal.tab", sep = "\t")

In [39]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/
grep -e "^##fileformat" -e "^##FORMAT" -e "^##INFO" \
    /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename.vcf \
    > /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped.vcf

In [40]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/
paste /storage/group/ibb3/default/convert_snpchip_coords/remapped_coords_apal.tab \
    <( grep -v "^##" /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename.vcf | cut -f4- ) \
    >> /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped.vcf

In [41]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/
grep "^#" /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped.vcf \
    > /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped_sorted.vcf
grep -v "^#" /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped.vcf \
    | grep -v "NOMATCH" \
    | sort -k1,1V -k2,2g \
    >> /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped_sorted.vcf

In [42]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/convert_snpchip_coords/
vcftools --vcf /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped_sorted.vcf \
    --min-alleles 2 --max-alleles 2 --max-missing 0.8 --mac 3 \
    --recode --out apal_mote_samples_rehead_filtered_remapped_sorted

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename_remapped_sorted.vcf
	--mac 3
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.8
	--out apal_mote_samples_rehead_filtered_remapped_sorted
	--recode

After filtering, kept 107 out of 107 Individuals
Outputting VCF file...
After filtering, kept 4093 out of a possible 20117 Sites
Run Time = 2.00 seconds


In [ ]:
mkdir -p /storage/group/ibb3/default/convert_snpchip_coords/lepwrap_apal
LepWrap 40 config_round1.yml

In [44]:
%%bash
cd /storage/group/ibb3/default/convert_snpchip_coords/lepwrap_apal/3_SeparateChromosomes/
cat ./all.LOD.summary | tr -s " " > tmp.txt

In [45]:
lg = pd.read_csv("/storage/group/ibb3/default/convert_snpchip_coords/lepwrap_apal/3_SeparateChromosomes/tmp.txt", sep = " ")

In [46]:
lg[lg.columns[4:]].head(50)

,LOD.4,LOD.5,LOD.6,LOD.7,LOD.8,LOD.9,LOD.10,LOD.11,LOD.12,LOD.13,LOD.14,LOD.15,LOD.16,LOD.17,LOD.18,LOD.19,LOD.20
0,965,965,966,966,966,967,967,976,977,990,992,1031,1080,1141,1154,1181,1203
1,566,357,357,357,357,357,357,357,350,348,348,329,274,100,74,59,59
2,363,207,207,207,207,207,207,205,205,205,205,199,128,99,68,49,49
3,357,198,198,198,198,198,198,198,198,193,193,173,87,75,62,44,44
4,194,194,194,194,194,194,194,194,194,190,190,170,81,74,56,43,43
5,170,189,189,189,189,189,189,186,186,174,174,153,79,68,51,41,41
6,164,174,174,174,174,174,174,174,174,170,170,142,77,64,49,40,40
7,152,170,170,170,170,170,170,170,170,170,169,140,76,62,44,39,39
8,140,164,164,164,164,164,164,164,164,163,163,134,75,60,43,39,39
9,138,161,161,161,161,161,161,157,157,156,156,130,68,56,43,38,38
